In [1]:
%run TAQplots.ipynb

In [2]:
OrderLog20020812_df = pd.read_csv('./resources/OrderLog20020812.txt')
OrderLog20020812_df = OrderLog20020812_df.astype({"NO":"int",
                                                  "SECCODE":"string",
                                                  "BUYSELL":"string",
                                                  "TIME":"string",
                                                  "ORDERNO":"int",
                                                  "ACTION":"int",
                                                  "PRICE":"float",
                                                  "VOLUME":"int",
                                                  "TRADENO":pd.Int64Dtype(),
                                                  "TRADEPRICE":"float"
                                                 })
OrderLog20020812_df.head()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,RU14VOST1005,S,103000000,1,1,2.380,4500,<NA>,NaN
1,2,RU0009071187,S,103000000,2,1,990.000,300,<NA>,NaN
2,3,RU14VOST1005,S,103000000,3,1,2.420,21600,<NA>,NaN
3,4,RU0009100762,B,103000000,4,1,0.133,100000,<NA>,NaN
4,5,RU0009054449,S,103000000,5,1,332.000,500,<NA>,NaN


In [4]:
dic = {4434:0, 'asdf':1}
dic[4434], dic['asdf'], len(dic)

(0, 1, 2)

In [5]:
class TAQ2TAQplots:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col, # orderno or some mark linking trades and quotes
              trdno_col
             ):
        self.taq_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.trdno_col = trdno_col
        self.secs = pd.unique(taq_df[sec_col])
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.trdno2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])}
        self.rows_to_edges()
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.TAQpps = {k:v for (k, v) in zip(self.secs, [TAQplots(self.Gs[sec]) for sec in self.secs])}

    def sec_rowid_lnk2edges(self, rowid, sec, lnk, trdno): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if trdno not in self.trdno2gkey_dic[sec]:
            self.trdno2gkey_dic[sec][trdno] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))
        if pd.notna(trdno):
            self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.trdno2gkey_dic[sec][trdno]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk, trdno) 
         for rowid, sec, lnk, trdno in zip(self.taq_df[self.row_id_col],
                                    self.taq_df[self.sec_col],
                                    self.taq_df[self.link_col],
                                    self.taq_df[self.trdno_col])]
        

In [13]:
t2tpp = TAQ2TAQplots(OrderLog20020812_df, 'NO', 'SECCODE', 'ORDERNO', 'TRADENO')
OrderLog20020812_df['TAQPLOTNO'] = [t2tpp.TAQpps[sec].TAQplotnos[t2tpp.rowid2gkey_dic[sec][rowid]] 
                                    for (sec, rowid) in zip(OrderLog20020812_df['SECCODE'], 
                                                            OrderLog20020812_df['NO'])]
OrderLog20020812_df.to_csv('./resources/OrderLog20020812_df.csv')
OrderLog20020812_df

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,TAQPLOTNO
0,1,RU14VOST1005,S,103000000,1,1,2.380,4500,<NA>,NaN,0
1,2,RU0009071187,S,103000000,2,1,990.000,300,<NA>,NaN,0
2,3,RU14VOST1005,S,103000000,3,1,2.420,21600,<NA>,NaN,2
3,4,RU0009100762,B,103000000,4,1,0.133,100000,<NA>,NaN,0
4,5,RU0009054449,S,103000000,5,1,332.000,500,<NA>,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
90739,90740,RU14TATN3006,B,184557000,38574,1,18.770,50000,<NA>,NaN,209
90740,90741,RU14VOST1005,S,184558000,38559,0,1.990,49800,<NA>,NaN,161
90741,90742,RU0009046700,S,184559000,38575,1,18.450,100000,<NA>,NaN,357
90742,90743,RU0009071187,B,184559000,38550,0,725.840,1,<NA>,NaN,230


In [9]:
OrderLog20020812_df[OrderLog20020812_df.TRADENO == 29846151]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,TAQPLOTNO
10713,10714,RU0009024277,B,110554000,5747,2,480.05,50,29846151,480.05,30
10714,10715,RU0009024277,S,110554000,6202,2,480.00,50,29846151,480.05,30


In [12]:
plotno_30_df = OrderLog20020812_df.loc[(OrderLog20020812_df.SECCODE == 'RU0009024277') & (OrderLog20020812_df.TAQPLOTNO == 30)]
plotno_30_df

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,TAQPLOTNO
480,481,RU0009024277,B,103000000,481,1,480.00,20,<NA>,NaN,30
1122,1123,RU0009024277,B,103000000,1123,1,480.02,200,<NA>,NaN,30
2155,2156,RU0009024277,B,103135000,1866,1,480.01,500,<NA>,NaN,30
3866,3867,RU0009024277,B,103701000,2834,1,480.01,50,<NA>,NaN,30
3880,3881,RU0009024277,B,103705000,2847,1,480.01,1,<NA>,NaN,30
...,...,...,...,...,...,...,...,...,...,...,...
13656,13657,RU0009024277,S,111707000,7550,2,480.00,138,29846687,480.0,30
13799,13800,RU0009024277,S,111737000,7624,1,480.00,100,<NA>,NaN,30
13801,13802,RU0009024277,B,111737000,2937,2,480.00,100,29846711,480.0,30
13802,13803,RU0009024277,S,111737000,7624,2,480.00,100,29846711,480.0,30
